# **Теория**

**AutoKeras** - это библиотека автоматического машинного обучения, разработанная для простого и быстрого создания высокопроизводительных моделей глубокого обучения. Она использует подход обучения без учителя для поиска оптимальной архитектуры модели и оптимальных гиперпараметров.

В библиотеке AutoKeras есть несколько тюнеров, которые могут быть использованы для автоматического подбора гиперпараметров моделей машинного обучения.

* **Random (random):** Случайный поиск является простейшим подходом к оптимизации гиперпараметров. Он просто выбирает случайные комбинации гиперпараметров из заданного распределения и обучает модель для каждой комбинации. Однако его главным недостатком является то, что он может быть менее эффективным, чем другие методы, особенно когда пространство гиперпараметров велико.

* **Bayesian (bayesian):** Байесовская оптимизация является более сложным и интеллектуальным подходом к оптимизации гиперпараметров. Он использует вероятностную модель для оценки качества различных комбинаций гиперпараметров. Вместо случайного выбора комбинаций гиперпараметров, байесовский тюнер выбирает их на основе предыдущих результатов и текущего состояния вероятностной модели.

* **Hyperband (hyperband):** Hyperband является адаптивным алгоритмом оптимизации гиперпараметров, основанным на идее адаптивного выделения ресурсов для различных комбинаций гиперпараметров. Вместо одинакового выделения ресурсов для каждой комбинации, Hyperband адаптируется на основе предыдущих результатов и выделяет больше ресурсов для комбинаций с большим потенциалом. Hyperband комбинирует две стратегии: диверсификация (начиная с множества случайных комбинаций и малого количества эпох) и интенсификация (продолжая обучение лучших комбинаций на большем количестве эпох). Это делает Hyperband более эффективным и адаптивным, чем случайный поиск или байесовская оптимизация. Однако, он также может быть более сложным и требовать больше вычислительных ресурсов, особенно на начальных этапах.

* **Greedy (greedy):** Жадный тюнер является методом последовательного выбора гиперпараметров. Он начинает с настройки базовых гиперпараметров, а затем последовательно настраивает каждый гиперпараметр, выбирая тот, который приводит к наибольшему улучшению результатов. Преимущества жадного подхода включают простоту и возможность быстрого улучшения результатов. Однако этот подход может привести к локальному оптимуму, так как он не учитывает возможные взаимодействия между гиперпараметрами.

# **Предобработка датасета**

In [1]:
# Установим библиотеку для работы с AutoKeras
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.5/584.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 92.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.4 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.14.0
    Uninstalling tensorflow-estimator-2.14.0:
      Successfully uninstalled tensorflow-estimator-2.14.0
  Attempting uninstall: keras
    Found existing install

In [2]:
# Импортируем необходимые библиотеки
import pandas as pd
from IPython.display import display
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import csv
import autokeras as ak
from sklearn.preprocessing import LabelEncoder
import pickle

Using TensorFlow backend


In [3]:
# получение csv-файла
google_url = 'https://docs.google.com/spreadsheets/d/1Rmw4nLN7cxRnhFFrIETh3gx8Gxcv56hV/edit#gid=548315443'
new_google_url = google_url.replace('edit#gid', 'export?format=csv&gid')
df=pd.read_csv(new_google_url)

In [4]:
# небольшая предобработка целевых столбцов (уверенность)
df['Поставьте галочку, если уверены на 100%'] = df['Поставьте галочку, если уверены на 100%'].replace(True, 'да')
df['Поставьте галочку, если уверены на 100%'] = df['Поставьте галочку, если уверены на 100%'].replace(False, 'нет')
df['Если 75%'] = df['Если 75%'].replace(True, 'да')
df['Если 75%'] = df['Если 75%'].replace(False, 'нет')
df['Если 50%'] = df['Если 50%'].replace(True, '50%')
df['Если 50%'] = df['Если 50%'].replace(False, 'нет')

df["Уверенность в диагнозе"] = df.apply(
    lambda row: "100%"
    if row["Поставьте галочку, если уверены на 100%"] == "да"
    else ("75%" if row["Если 75%"] == "да" else "50%"), axis=1)

df = df.drop(columns=["Поставьте галочку, если уверены на 100%", "Если 75%", "Если 50%"])
df

,id_ответа,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,Ко-мутации KEAP1,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию,Ответ эксперта (Лактионов),"Альтернатива, если 50% - обязательно",Комментарий,Уверенность в диагнозе
0,1,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,до 42 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,0,0,100%
1,2,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,от 43 до 60 дней,нет мутаций,Не исследовался,Результативность лечения,Дурвалумаб,0,0,100%
2,3,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,более 61 дня,нет мутаций,Не исследовался,Результативность лечения,Наблюдение,Дурвалумаб,Нет доказательной базы для назначения после пе...,75%
3,4,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,до 42 дней,не исследовались,Не исследовался,Результативность лечения,Дурвалумаб,0,Возможно есть мутации и эффективность Дурвалум...,75%
4,5,Азиатская,Мужской,До 70,В настоящее время,0-1,да,да,да,да,да,от 43 до 60 дней,не исследовались,Не исследовался,Результативность лечения,Дурвалумаб,0,Возможно есть мутации и эффективность Дурвалум...,75%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248827,248828,Другая,Женский,>70,Курение в прошлом (бросил более 1 года),2,нет,нет,нет,нет,нет,от 43 до 60 дней,EGFR ex21,Более 1%,сохранение качества жизни,Наблюдение,Осимертиниб,"Возможна низкая эффективность Дурвалумаба, Оси...",50%
248828,248829,Другая,Женский,>70,Курение в прошлом (бросил более 1 года),2,нет,нет,нет,нет,нет,более 61 дня,EGFR ex21,Более 1%,сохранение качества жизни,Наблюдение,0,0,100%
248829,248830,Другая,Женский,>70,Курение в прошлом (бросил более 1 года),2,нет,нет,нет,нет,нет,до 42 дней,ALK позитивный,Более 1%,сохранение качества жизни,Алектиниб,Дурвалумаб,"Возможна низкая эффективность Дурвалумаба, Але...",50%
248830,248831,Другая,Женский,>70,Курение в прошлом (бросил более 1 года),2,нет,нет,нет,нет,нет,от 43 до 60 дней,ALK позитивный,Более 1%,сохранение качества жизни,Алектиниб,Дурвалумаб,"Возможна низкая эффективность Дурвалумаба, Але...",50%


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 248832 entries, 0 to 248831
Data columns (total 19 columns):
 #   Column                                                    Non-Null Count   Dtype 
---  ------                                                    --------------   ----- 
 0   id_ответа                                                 248832 non-null  int64 
 1   Раса                                                      248832 non-null  object
 2   Пол                                                       248832 non-null  object
 3   Возраст                                                   248832 non-null  object
 4   Статус курения                                            248832 non-null  object
 5   ECOG                                                      248832 non-null  object
 6   Есть опухолевая нагрузка? (симптомная опухоль)            248832 non-null  object
 7   Ко-мутации KRAS                                           248832 non-null  object
 8   Ко-мутации p53

In [6]:
# Создание экземпляра LabelEncoder
label_encoder = LabelEncoder()

# Применение LabelEncoder ко всем столбцам
for col in df.columns:
    if df[col].dtype == 'object':  # Проверка, что столбец имеет тип 'object'
        df[col] = label_encoder.fit_transform(df[col].astype(str))

In [7]:
# Отделим входные переменные
x = df[df.columns[1:15]]
x

,Раса,Пол,Возраст,Статус курения,ECOG,Есть опухолевая нагрузка? (симптомная опухоль),Ко-мутации KRAS,Ко-мутации p53.,Ко-мутации STK11,Ко-мутации KEAP1,Срок от окончания ХЛТ,Молекулярный статус (только для неплоскоклеточного рака),PD-L1 статус,Предпочтение пациента по ответу на терапию
0,0,1,1,0,0,0,0,0,0,0,1,5,2,0
1,0,1,1,0,0,0,0,0,0,0,2,5,2,0
2,0,1,1,0,0,0,0,0,0,0,0,5,2,0
3,0,1,1,0,0,0,0,0,0,0,1,4,2,0
4,0,1,1,0,0,0,0,0,0,0,2,4,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248827,1,0,0,1,1,1,1,1,1,1,2,2,0,1
248828,1,0,0,1,1,1,1,1,1,1,0,2,0,1
248829,1,0,0,1,1,1,1,1,1,1,1,0,0,1
248830,1,0,0,1,1,1,1,1,1,1,2,0,0,1


In [8]:
# Выходная переменная
y = df[df.columns[15:19]]
y

,Ответ эксперта (Лактионов),"Альтернатива, если 50% - обязательно",Комментарий,Уверенность в диагнозе
0,1,0,0,0
1,1,0,0,0
2,2,2,10,2
3,1,0,7,2
4,1,0,7,2
...,...,...,...,...
248827,2,4,6,1
248828,2,0,0,0
248829,0,2,4,1
248830,0,2,4,1


In [9]:
# Разбиение наборов на общую и тестовую выборки
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True, random_state = 42)

In [10]:
# Вывод размеров обучающей и проверочной выборок
print(f'Размер обучающей выборки: {x_train.shape[0]}')
print(f'Размер проверочной выборки: {x_test.shape[0]}')

Размер обучающей выборки: 223948
Размер проверочной выборки: 24884


# **Эксперимент 1. Tuner RandomSearch, 10 запусков**

In [ ]:
# инициализация класса автоматического машинного обучения
reg = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=10,
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment1',
)

# поиск лучшей модели
reg.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))

Trial 10 Complete [00h 06m 55s]
val_loss: 0.034590449184179306

Best val_loss So Far: 2.0720716809030648e-10
Total elapsed time: 01h 03m 40s
Epoch 1/10
6999/6999 [==============================] - 25s 3ms/step - loss: 1.8512 - mean_absolute_error: 0.8656 - val_loss: 0.0558 - val_mean_absolute_error: 0.1698
Epoch 2/10
6999/6999 [==============================] - 22s 3ms/step - loss: 0.0078 - mean_absolute_error: 0.0524 - val_loss: 0.0012 - val_mean_absolute_error: 0.0235
Epoch 3/10
6999/6999 [==============================] - 22s 3ms/step - loss: 0.0019 - mean_absolute_error: 0.0235 - val_loss: 3.0316e-04 - val_mean_absolute_error: 0.0133
Epoch 4/10
6999/6999 [==============================] - 22s 3ms/step - loss: 0.0014 - mean_absolute_error: 0.0193 - val_loss: 4.3710e-05 - val_mean_absolute_error: 0.0043
Epoch 5/10
6999/6999 [==============================] - 23s 3ms/step - loss: 9.1365e-04 - mean_absolute_error: 0.0152 - val_loss: 2.1405e-04 - val_mean_absolute_error: 0.0097
Epoch 6/

In [ ]:
best_model_1 = reg.export_model()
#best_model_1.save('best_model_1.h5')
#best_model_1.save('best_model_1_savedmodel', save_format='tf')
with open('best_model_1.pkl', 'wb') as f:
    pickle.dump(best_model_1, f)
# Выведите архитектуру модели
print("Model architecture:")
print(best_model_1.summary())

Model architecture:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 multi_category_encoding (M  (None, 14)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 14)                29        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                480       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 512) 

# **Эксперимент №2. Tuner Hyperband, 10 запусков**

In [ ]:
# инициализация класса автоматического машинного обучения
reg2 = ak.StructuredDataRegressor(
    overwrite=True,
    max_trials=10,
    tuner="hyperband",
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment2'
)

# поиск лучшей модели
reg2.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

Trial 10 Complete [00h 03m 09s]
val_loss: 2.493952512741089

Best val_loss So Far: 0.21631589531898499
Total elapsed time: 00h 32m 23s
Epoch 1/20
6999/6999 [==============================] - 22s 3ms/step - loss: 2.1582 - mean_absolute_error: 1.0242 - val_loss: 0.9788 - val_mean_absolute_error: 0.6724
Epoch 2/20
6999/6999 [==============================] - 21s 3ms/step - loss: 0.6473 - mean_absolute_error: 0.5495 - val_loss: 0.1390 - val_mean_absolute_error: 0.2662
Epoch 3/20
6999/6999 [==============================] - 21s 3ms/step - loss: 0.2637 - mean_absolute_error: 0.3629 - val_loss: 0.0871 - val_mean_absolute_error: 0.2138
Epoch 4/20
6999/6999 [==============================] - 21s 3ms/step - loss: 0.2041 - mean_absolute_error: 0.3171 - val_loss: 0.0479 - val_mean_absolute_error: 0.1550
Epoch 5/20
6999/6999 [==============================] - 22s 3ms/step - loss: 0.1836 - mean_absolute_error: 0.3003 - val_loss: 0.0283 - val_mean_absolute_error: 0.1227
Epoch 6/20
6999/6999 [========

In [ ]:
best_model_2 = reg.export_model()
with open('best_model_2.pkl', 'wb') as f:
    pickle.dump(best_model_2, f)
# Выведите архитектуру модели
print("Model architecture:")
print(best_model_2.summary())

Model architecture:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 multi_category_encoding (M  (None, 14)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 14)                29        
 on)                                                             
                                                                 
 dense (Dense)               (None, 32)                480       
                                                                 
 re_lu (ReLU)                (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 512) 

# **Эксперимент №3. Tuner Greedy, 10 запусков**

In [ ]:
# создание и обучение модели
reg3 = ak.AutoModel(
    inputs=ak.StructuredDataInput(),
    outputs=ak.RegressionHead(),
    max_trials=10,
    tuner="greedy",
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment3',
    overwrite=True)
reg3.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

# оценка модели
results = reg3.evaluate(x_test, y_test)
print(results)

Trial 10 Complete [00h 15m 19s]
val_loss: 0.00010737845150288194

Best val_loss So Far: 0.00010737845150288194
Total elapsed time: 02h 05m 44s
Epoch 1/20
6999/6999 [==============================] - 33s 4ms/step - loss: 0.2952 - mean_absolute_error: 0.2532 - val_loss: 0.0076 - val_mean_absolute_error: 0.0638
Epoch 2/20
6999/6999 [==============================] - 29s 4ms/step - loss: 0.0082 - mean_absolute_error: 0.0599 - val_loss: 0.0083 - val_mean_absolute_error: 0.0689
Epoch 3/20
6999/6999 [==============================] - 31s 4ms/step - loss: 0.0049 - mean_absolute_error: 0.0478 - val_loss: 0.0242 - val_mean_absolute_error: 0.1110
Epoch 4/20
6999/6999 [==============================] - 28s 4ms/step - loss: 0.0037 - mean_absolute_error: 0.0396 - val_loss: 0.0012 - val_mean_absolute_error: 0.0261
Epoch 5/20
6999/6999 [==============================] - 31s 4ms/step - loss: 0.0032 - mean_absolute_error: 0.0356 - val_loss: 0.0034 - val_mean_absolute_error: 0.0399
Epoch 6/20
6999/6999 [

In [ ]:
best_model_3 = reg3.export_model()
with open('best_model_3.pkl', 'wb') as f:
    pickle.dump(best_model_3, f)
# Выведите архитектуру модели
print("Model architecture:")
print(best_model_3.summary())

Model architecture:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 multi_category_encoding (M  (None, 14)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 14)                29        
 on)                                                             
                                                                 
 dense (Dense)               (None, 1024)              15360     
                                                                 
 re_lu (ReLU)                (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 64)  

# **Эксперимент №4. Tuner Bayesian, 10 запусков**

In [11]:
# создание и обучение модели
reg4 = ak.AutoModel(
    inputs=ak.StructuredDataInput(),
    outputs=ak.RegressionHead(),
    max_trials=10,
    tuner="bayesian",
    loss='mean_squared_error',
    metrics=['mean_absolute_error'],
    project_name='experiment4',
    overwrite=True)
reg4.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

# оценка модели
results = reg4.evaluate(x_test, y_test)
print(results)

Trial 10 Complete [00h 08m 23s]
val_loss: 2.158161163330078

Best val_loss So Far: 0.0007164972485043108
Total elapsed time: 01h 22m 16s
Epoch 1/20
6999/6999 [==============================] - 17s 2ms/step - loss: 0.1585 - mean_absolute_error: 0.1826 - val_loss: 0.0088 - val_mean_absolute_error: 0.0653
Epoch 2/20
6999/6999 [==============================] - 16s 2ms/step - loss: 0.0185 - mean_absolute_error: 0.0793 - val_loss: 0.0070 - val_mean_absolute_error: 0.0599
Epoch 3/20
6999/6999 [==============================] - 17s 2ms/step - loss: 0.0127 - mean_absolute_error: 0.0623 - val_loss: 0.0136 - val_mean_absolute_error: 0.0777
Epoch 4/20
6999/6999 [==============================] - 16s 2ms/step - loss: 0.0100 - mean_absolute_error: 0.0522 - val_loss: 0.0300 - val_mean_absolute_error: 0.1201
Epoch 5/20
6999/6999 [==============================] - 16s 2ms/step - loss: 0.0098 - mean_absolute_error: 0.0512 - val_loss: 0.0391 - val_mean_absolute_error: 0.1262
Epoch 6/20
6999/6999 [======

In [12]:
best_model_4 = reg4.export_model()
with open('best_model_4.pkl', 'wb') as f:
    pickle.dump(best_model_4, f)
# Выведите архитектуру модели
print("Model architecture:")
print(best_model_4.summary())

Model architecture:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 multi_category_encoding (M  (None, 14)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 14)                29        
 on)                                                             
                                                                 
 dense (Dense)               (None, 64)                960       
                                                                 
 re_lu (ReLU)                (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)  